In [ ]:
# import time
# from scapy.all import rdpcap
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import IsolationForest

# # Global dictionary to hold active flows
# active_flows = {}

# # Function to update flows
# def update_flows(new_packets):
#     for pkt in new_packets:
#         if pkt.haslayer('IP'):
#             ip = pkt['IP']
#             if pkt.haslayer('TCP') or pkt.haslayer('UDP'):
#                 sport = pkt.sport
#                 dport = pkt.dport
#             else:
#                 sport, dport = 0, 0
#             key = (ip.src, sport, ip.dst, dport, ip.proto)
#             ts = pkt.time
#             pkt_len = len(pkt)
#             ttl = ip.ttl
#             tcp_flags = pkt.sprintf("%TCP.flags%") if pkt.haslayer("TCP") else ""
#             tcp_win = pkt['TCP'].window if pkt.haslayer("TCP") else None
#             tcp_seq = pkt['TCP'].seq if pkt.haslayer("TCP") else None
            
#             if key not in active_flows:
#                 active_flows[key] = {
#                     'ts_start': ts,
#                     'ts_end': ts,
#                     'orig_bytes': pkt_len,
#                     'resp_bytes': 0,
#                     'orig_pkts': 1,
#                     'resp_pkts': 0,
#                     'ttl_orig': [ttl],
#                     'ttl_resp': [],
#                     'timestamps_orig': [ts],
#                     'timestamps_resp': [],
#                     'win_orig': [tcp_win] if tcp_win is not None else [],
#                     'win_resp': [tcp_win] if tcp_win is not None else [],
#                     'seq_orig': tcp_seq if tcp_seq is not None else None,
#                     'seq_resp': None,
#                 }
#             else:
#                 flow = active_flows[key]
#                 flow['ts_end'] = ts
#                 if ip.src == key[0]:
#                     flow['orig_bytes'] += pkt_len
#                     flow['orig_pkts'] += 1
#                     flow['ttl_orig'].append(ttl)
#                     flow['timestamps_orig'].append(ts)
#                     if tcp_win is not None:
#                         flow['win_orig'].append(tcp_win)
#                 else:
#                     flow['resp_bytes'] += pkt_len
#                     flow['resp_pkts'] += 1
#                     flow['ttl_resp'].append(ttl)
#                     flow['timestamps_resp'].append(ts)
#                     if tcp_win is not None:
#                         flow['win_resp'].append(tcp_win)
#                     if flow['seq_resp'] is None and tcp_seq is not None:
#                         flow['seq_resp'] = tcp_seq

# # Function to extract features from completed flows
# def extract_features_from_completed_flows(inactivity_threshold=5):
#     current_time = time.time()
#     completed = []
#     keys_to_remove = []
    
#     for key, flow in active_flows.items():
#         if current_time - flow['ts_end'] > inactivity_threshold:
#             features = extract_features_from_flow(flow)
#             features.update({
#                 'src': key[0],
#                 'sport': key[1],
#                 'dst': key[2],
#                 'dport': key[3],
#                 'proto': key[4]
#             })
#             completed.append(features)
#             keys_to_remove.append(key)
    
#     for key in keys_to_remove:
#         del active_flows[key]
    
#     return pd.DataFrame(completed)

# # Function to extract features from a single flow
# def extract_features_from_flow(flow):
#     duration = flow['ts_end'] - flow['ts_start']
#     orig_bytes = flow['orig_bytes']
#     resp_bytes = flow['resp_bytes']
#     orig_pkts = flow['orig_pkts']
#     resp_pkts = flow['resp_pkts']
    
#     avg_ttl_orig = np.mean(flow['ttl_orig']) if flow['ttl_orig'] else 0
#     avg_ttl_resp = np.mean(flow['ttl_resp']) if flow['ttl_resp'] else 0
#     total_pkts = orig_pkts + resp_pkts
#     overall_avg_ttl = (np.sum(flow['ttl_orig']) + np.sum(flow['ttl_resp'])) / total_pkts if total_pkts > 0 else 0
    
#     mean_interpkt_orig, std_interpkt_orig = compute_interpacket(flow['timestamps_orig'])
#     mean_interpkt_resp, std_interpkt_resp = compute_interpacket(flow['timestamps_resp'])
    
#     Sload = (orig_bytes * 8) / duration if duration > 0 else 0
#     Dload = (resp_bytes * 8) / duration if duration > 0 else 0
    
#     mean_pkt_size_orig = orig_bytes / orig_pkts if orig_pkts > 0 else 0
#     mean_pkt_size_resp = resp_bytes / resp_pkts if resp_pkts > 0 else 0
#     load_diff = Sload - Dload
    
#     swin = np.mean(flow['win_orig']) if flow['win_orig'] else 0
#     dwin = np.mean(flow['win_resp']) if flow['win_resp'] else 0
#     stcpb = flow['seq_orig'] if flow['seq_orig'] is not None else 0
#     dtcpb = flow['seq_resp'] if flow['seq_resp'] is not None else 0
    
#     is_sm_ips_ports = 1 if (flow.get('src') == flow.get('dst')) else 0 
    
#     features = {
#         'duration': duration,
#         'orig_bytes': orig_bytes,
#         'resp_bytes': resp_bytes,
#         'orig_pkts': orig_pkts,
#         'resp_pkts': resp_pkts,
#         'overall_avg_ttl': overall_avg_ttl,
#         'mean_interpkt_orig': mean_interpkt_orig,
#         'std_interpkt_orig': std_interpkt_orig,
#         'mean_interpkt_resp': mean_interpkt_resp,
#         'std_interpkt_resp': std_interpkt_resp,
#         'Sload': Sload,
#         'Dload': Dload,
#         'mean_pkt_size_orig': mean_pkt_size_orig,
#         'mean_pkt_size_resp': mean_pkt_size_resp,
#         'load_diff': load_diff,
#         'swin': swin,
#         'dwin': dwin,
#         'stcpb': stcpb,
#         'dtcpb': dtcpb,
#         'is_sm_ips_ports': is_sm_ips_ports
#     }
#     return features

# # Real-time simulation
# iso_forest = IsolationForest(contamination=0.01, random_state=42)
# model_trained = False

# for iteration in range(5):
#     print(f"Iteration {iteration + 1} - Capturing new packets...")
#     new_packets = rdpcap('captured_traffic3.pcap', count=200)
#     update_flows(new_packets)
    
#     df_completed = extract_features_from_completed_flows(inactivity_threshold=5)
    
#     if not df_completed.empty:
#         print("Extracted completed flow features:")
#         display(df_completed.head())
        
#         if not model_trained:
#             X_train = df_completed[selected_features].fillna(0)
#             iso_forest.fit(X_train)
#             model_trained = True
#             print("Model trained on first batch.")
        
#         X_live = df_completed[selected_features].fillna(0)
#         predictions = iso_forest.predict(X_live)
#         df_completed['anomaly'] = predictions
        
#         anomalies = df_completed[df_completed['anomaly'] == -1]
#         if not anomalies.empty:
#             print("🚨 ALERT: Anomalous flow detected!")
#             display(anomalies)
#     else:
#         print("No completed flows in this iteration.")
    
#     time.sleep(2)

# print("Real-time simulation loop completed.")


#!/usr/bin/env python
from scapy.all import sniff, IP, TCP, UDP
import pandas as pd
import time

# Dictionary to store flows
flows = {}

def extract_features(flow_key):
    """Convert flow data into a feature vector."""
    flow = flows[flow_key]
    return {
        'packet_count': flow.get('packet_count', 0),
        'byte_count': flow.get('byte_count', 0),
        'sbytes': flow.get('sbytes', 0),
        'dbytes': flow.get('dbytes', 0),
        'spkts': flow.get('spkts', 0),
        'dpkts': flow.get('dpkts', 0),
        'sload': flow.get('sload', 0),
        'dload': flow.get('dload', 0),
        'source_ttl': flow.get('source_ttl', 0),
        'dest_ttl': flow.get('dest_ttl', 0),
        'source_tcp_win': flow.get('source_tcp_win', 0),
        'dest_tcp_win': flow.get('dest_tcp_win', 0),
        'tcprtt': flow.get('tcprtt', 0),
        'http_methods': flow.get('http_methods', 0),
        'ftp_cmds': flow.get('ftp_cmds', 0)
    }

def process_packet(packet):
    """Processes packets and extracts network traffic features."""
    if IP in packet:
        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        proto = packet[IP].proto
        timestamp = time.time()
        src_port, dst_port, pkt_size, win_size = None, None, len(packet), None

        if TCP in packet:
            src_port, dst_port = packet[TCP].sport, packet[TCP].dport
            win_size = packet[TCP].window
        elif UDP in packet:
            src_port, dst_port = packet[UDP].sport, packet[UDP].dport

        flow_key = (src_ip, dst_ip, src_port, dst_port, proto)

        # Update flow statistics
        if flow_key not in flows:
            flows[flow_key] = {
                'start_time': timestamp,
                'packet_count': 0, 'byte_count': 0, 'sbytes': 0, 'dbytes': 0,
                'spkts': 0, 'dpkts': 0, 'sload': 0, 'dload': 0,
                'source_ttl': packet[IP].ttl, 'dest_ttl': None,
                'source_tcp_win': win_size, 'dest_tcp_win': None,
                'tcprtt': None, 'http_methods': 0, 'ftp_cmds': 0
            }

        flow = flows[flow_key]
        flow['packet_count'] += 1
        flow['byte_count'] += pkt_size
        if src_port:
            flow['sbytes'] += pkt_size
            flow['spkts'] += 1
        if dst_port:
            flow['dbytes'] += pkt_size
            flow['dpkts'] += 1

        # Compute traffic rates
        duration = timestamp - flow['start_time']
        if duration > 0:
            flow['sload'] = (flow['sbytes'] * 8) / duration
            flow['dload'] = (flow['dbytes'] * 8) / duration

def save_to_csv():
    """Saves network flows to a CSV file."""
    df = pd.DataFrame([extract_features(flow) for flow in flows])
    df.to_csv("network_profile.csv", index=False)
    print("✅ Network profile saved to network_profile.csv")

def capture_packets(interface="Wi-Fi", count=100):
    """Captures network packets and extracts features."""
    print(f"🚀 Capturing {count} packets on {interface} and saving to network_profile.csv...")
    sniff(iface=interface, prn=process_packet, count=count, store=False)
    save_to_csv()

if __name__ == "__main__":
    capture_packets(interface="Wi-Fi", count=100)


Iteration 1 - Capturing new packets...


NameError: name 'compute_interpacket' is not defined

Iteration 1 - Capturing new packets...


NameError: name 'extract_features_from_completed_flows' is not defined